In [1]:
from microscope import *
import ipywidgets as widgets

%matplotlib inline

In [11]:
l_1 = 1064e-9
l_2 = 532e-9
N_POINTS = 256  # Resolution of image
pixel_size = 1e-10

input_coordinate_system = CoordinateSystem(dxdydz=(pixel_size, pixel_size), n_points=(N_POINTS, N_POINTS))
first_wave = WaveFunction(psi=np.ones((N_POINTS, N_POINTS)),
                          coordinates=input_coordinate_system,
                          E0=Joules_of_keV(300))

dummy_sample = SamplePropagator(dummy_potential=f'letters_{N_POINTS}',
                                coordinates_for_dummy_potential=CoordinateSystem(axes=(input_coordinate_system.x_axis,
                                                                                       input_coordinate_system.y_axis,
                                                                                       np.linspace(-5e-10, 5e-10, 2)
                                                                                       )))

first_lens = LensPropagator(focal_length=3.3e-3, fft_shift=True)
second_lens = LensPropagator(focal_length=3.3e-3, fft_shift=False)

E_1_default = find_amplitude_for_phase(starting_E=1e7, cavity_type='analytical', print_progress=False, NA_1=0.05)

In [15]:
def f(NA_1, defocus_nm, Cs_nm, n_electrons, ring_cavity, E_1):
    cavity = CavityAnalyticalPropagator(l_1=l_1, l_2=l_2, E_1=E_1, NA_1=NA_1)
    aberration_propagator = AberrationsPropagator(Cs=Cs_nm*1e-9, defocus=defocus_nm*1e-9, astigmatism_parameter=0, astigmatism_orientation=0)
    
    M_2f_a = Microscope([dummy_sample, first_lens, cavity, second_lens, aberration_propagator], n_electrons=n_electrons)
    pic_2f_a = M_2f_a.take_a_picture(first_wave)
    
#     fig, ax = plt.subplots(1, 1, figsize=(14, 10))
    fig = plt.figure(figsize=(14, 10))
    plt.imshow(pic_2f_a.values)  # , extent=pic_2f_a.coordinates.limits
    plt.colorbar()
    
    plt.xlabel("y")
    plt.ylabel("x")
    

widgets.interact(f, NA_1=(0.001, 0.2, 0.01), defocus_nm=(-10, 10, 0.1), Cs_nm=(0, 100, 1), n_electrons=(0, 1e8, 1e7), ring_cavity=True, E_1=(E_1_default / 5, E_1_default * 5, E_1_default / 100));

interactive(children=(FloatSlider(value=0.091, description='NA_1', max=0.2, min=0.001, step=0.01), FloatSlider…

In [7]:
l_1 = 1064e-9
l_2 = 532e-9
N_POINTS = 256  # Resolution of image
pixel_size = 1e-10

input_coordinate_system = CoordinateSystem(dxdydz=(pixel_size, pixel_size), n_points=(N_POINTS, N_POINTS))
first_wave = WaveFunction(psi=np.ones((N_POINTS, N_POINTS)),
                          coordinates=input_coordinate_system,
                          E0=Joules_of_keV(300))

dummy_sample = SamplePropagator(dummy_potential=f'letters_{N_POINTS}',
                                coordinates_for_dummy_potential=CoordinateSystem(axes=(input_coordinate_system.x_axis,
                                                                                       input_coordinate_system.y_axis,
                                                                                       np.linspace(-5e-10, 5e-10, 2)
                                                                                       )))
E_1_default = find_amplitude_for_phase(starting_E=1e7, cavity_type='analytical', print_progress=False, NA_1=0.05)
first_lens = LensPropagator(focal_length=3.3e-3, fft_shift=True)
second_lens = LensPropagator(focal_length=3.3e-3, fft_shift=False)

E_1_default = find_amplitude_for_phase(starting_E=1e7, cavity_type='analytical', print_progress=False, NA_1=0.05)

def f(NA_1, defocus, Cs, n_electrons, ring_cavity, E_1):
    cavity = CavityNumericalPropagator(l_1=l_1, l_2=l_2, E_1=-1, NA_1=NA_1)
    aberration_propagator = AberrationsPropagator(Cs=Cs, defocus=defocus, astigmatism_parameter=0, astigmatism_orientation=0)
    
    M_2f_a = Microscope([dummy_sample, first_lens, cavity, second_lens, aberration_propagator], n_electrons=n_electrons)
    pic_2f_a = M_2f_a.take_a_picture(first_wave)
    
#     fig, ax = plt.subplots(1, 1, figsize=(14, 10))
    fig = plt.figure(figsize=(14, 10))
    plt.imshow(pic_2f_a.values)  # , extent=pic_2f_a.coordinates.limits
    plt.colorbar()
    
    plt.xlabel("y")
    plt.ylabel("x")
    

widgets.interact(f, NA_1=[0.03, 0.05, 0.08, 0.1, 0.15, 0.2], defocus=(-1e-8, 1e-8, 3e-10), Cs=(0, 1e-7, 1e-9), n_electrons=(0, 1e8, 1e7), ring_cavity=True, E_1=(E_1_default / 5, E_1_default * 5, E_1_default / 100));

interactive(children=(Dropdown(description='NA_1', options=(0.03, 0.05, 0.08, 0.1, 0.15, 0.2), value=0.03), Fl…